<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/474_MO_data_loading_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Why This Design Builds Trust and Control

This module embodies several non-negotiable enterprise principles:

* **Configurability**: Missions change without code changes
* **Auditability**: Inputs are inspectable and versionable
* **Predictability**: No hidden logic or implicit behavior
* **Governance-ready**: Decisions are explainable after the fact

Executives don’t need to understand Python to understand this system — because the *rules are visible in data*.

---

## How This Fits Into the Mission Orchestrator

This module feeds the orchestrator everything it needs to:

* Understand the mission
* Know the plan
* Select the right agents
* Measure success
* Contextualize decisions

It deliberately does **nothing else**.

That restraint is a strength — it keeps responsibilities clear and the system maintainable.

---


# Data Loading Utilities — Architectural Explanation

## What This Component Does

This module is the **data ingestion and normalization layer** for the Mission Orchestrator Agent.

Its responsibility is simple but critical:
**load all mission-defining inputs from configuration files and present them to the orchestrator in a clean, predictable structure.**

Rather than hard-coding business logic, tasks, KPIs, or agent definitions, this design treats them as **external, configurable data**. The orchestrator reads the “rules of the mission” rather than embedding them.

In practice, this means:

* Business intent lives outside the code
* Strategy can change without redeploying systems
* The orchestrator remains stable as missions evolve

This is a foundational design decision that enables transparency, auditability, and executive control.

---

## Why This Matters at the System Level

Most AI systems fail to scale because:

* Logic is buried in code
* Behavior changes require engineering effort
* Leaders can’t see or adjust how decisions are made

This module prevents those failures by enforcing a clear separation between:

* **Execution logic** (the orchestrator)
* **Business configuration** (missions, tasks, KPIs, agents)

That separation is what allows this agent to behave like **enterprise infrastructure**, not a prototype.

---

## Key Capabilities Provided by This Module

### 1. Mission as a First-Class Concept

The orchestrator does not operate on vague goals.
It loads a **specific mission definition** identified by `mission_id`.

This ensures:

* Every run has a clear objective
* Missions are repeatable and comparable
* Progress can be measured consistently

From a leadership perspective, this enables questions like:

> “Which missions are active?”
> “What is this system trying to accomplish right now?”

---

### 2. Explicit Task Decomposition

Rather than letting an AI model decide “what to do next,” tasks are:

* Predefined
* Structured
* Loaded from configuration

The orchestrator receives a **decomposed mission plan**, not an open-ended instruction.

This:

* Reduces unpredictability
* Improves explainability
* Makes execution auditable

It also mirrors how real organizations operate — leadership defines *what must happen*, not just *what they hope will happen*.

---

### 3. KPI-Driven Execution

KPIs are loaded as data, not inferred dynamically.

This ensures:

* Success criteria are known before execution begins
* Metrics reflect business priorities, not model behavior
* Performance can be evaluated objectively

This design is essential for ROI tracking and executive reporting, because outcomes are measured against **pre-agreed definitions of success**.

---

### 4. Controlled Agent Selection via Capabilities

Specialized agents are treated as **resources with declared capabilities**, not magical generalists.

By loading:

* A registry of available agents
* A task-to-agent capability matrix

…the orchestrator can make controlled, explainable routing decisions.

This prevents:

* Arbitrary agent assignment
* Hidden decision logic
* Over-reliance on any single component

From a governance standpoint, this allows leadership to ask:

> “Which agents are allowed to perform which tasks — and why?”

---

### 5. Fast, Transparent Lookups

Lookup dictionaries are built deliberately to support:

* Deterministic behavior
* Predictable performance
* Clear traceability

Rather than complex inference, the system relies on **explicit mappings** between tasks and agents.

This reinforces your core philosophy:

> The system should explain what it does — not surprise anyone.

---

### 6. Optional Execution Context

The execution context is treated as **supplemental input**, not a requirement.

This allows:

* Historical data to influence execution
* Context-aware decisions when available
* Graceful operation when context is missing

Importantly, the system does not fail or hallucinate if context is absent — it proceeds based on known rules.

This is a subtle but powerful reliability feature.

---

### 7. Standardized Result Templates

Agent outputs are expected to follow **predefined result structures**.

This:

* Enforces consistency across agents
* Simplifies downstream evaluation
* Makes performance comparison possible

Rather than parsing unpredictable responses, the orchestrator knows what “completion” looks like for each task.

This is a prerequisite for scalable evaluation, monitoring, and ROI analysis.

---


## Bottom Line

This data loading layer is not “just utilities.”

It is the **contract between business intent and system execution**.

By externalizing missions, tasks, KPIs, agent capabilities, and expected outcomes, you’ve created an orchestrator that:

* Can scale across missions
* Can be governed by leadership
* Can evolve without fragility
* Can justify its decisions

This is exactly the kind of foundational work that makes the rest of the agent credible.



In [ ]:
"""Data Loading Utilities

Load mission data, tasks, agents, KPIs, and execution context from JSON files.
"""

import json
from pathlib import Path
from typing import Dict, Any, List, Optional


def load_json_file(file_path: str) -> Any:
    """Load JSON file and return parsed data"""
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    with open(path, 'r') as f:
        return json.load(f)


def load_mission_data(mission_id: str, data_dir: str = "agents/data") -> Dict[str, Any]:
    """Load mission data by mission_id"""
    file_path = Path(data_dir) / "business_missions.json"
    missions = load_json_file(str(file_path))

    mission = next(
        (m for m in missions if m.get("mission_id") == mission_id),
        None
    )

    if not mission:
        raise ValueError(f"Mission {mission_id} not found")

    return mission


def load_mission_tasks(mission_id: str, data_dir: str = "agents/data") -> List[Dict[str, Any]]:
    """Load all tasks for a specific mission"""
    file_path = Path(data_dir) / "decomposed_mission_tasks.json"
    all_mission_tasks = load_json_file(str(file_path))

    mission_data = next(
        (m for m in all_mission_tasks if m.get("mission_id") == mission_id),
        None
    )

    if not mission_data:
        raise ValueError(f"Tasks for mission {mission_id} not found")

    return mission_data.get("tasks", [])


def load_mission_kpis(mission_id: str, data_dir: str = "agents/data") -> Dict[str, Any]:
    """Load KPIs for a specific mission"""
    file_path = Path(data_dir) / "mission_kpis.json"
    all_kpis = load_json_file(str(file_path))

    kpi_data = next(
        (k for k in all_kpis if k.get("mission_id") == mission_id),
        None
    )

    if not kpi_data:
        raise ValueError(f"KPIs for mission {mission_id} not found")

    return kpi_data.get("kpis", {})


def load_specialized_agents(data_dir: str = "agents/data") -> List[Dict[str, Any]]:
    """Load all specialized agents"""
    file_path = Path(data_dir) / "specialized_agents.json"
    return load_json_file(str(file_path))


def load_capabilities_matrix(data_dir: str = "agents/data") -> List[Dict[str, Any]]:
    """Load task-to-agent capabilities matrix"""
    file_path = Path(data_dir) / "agent_capabilities_matrix.json"
    return load_json_file(str(file_path))


def build_agent_lookup(agents: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Create fast lookup dictionary for agents by agent_id"""
    return {agent.get("agent_id"): agent for agent in agents}


def build_capabilities_lookup(capabilities_matrix: List[Dict[str, Any]]) -> Dict[str, List[str]]:
    """Create fast lookup dictionary: task_id -> list of capable agent_ids"""
    lookup = {}
    for entry in capabilities_matrix:
        task_id = entry.get("task_id")
        capable_agents = entry.get("capable_agents", [])
        if task_id:
            lookup[task_id] = capable_agents
    return lookup


def load_execution_context(mission_id: str, data_dir: str = "agents/data") -> Optional[Dict[str, Any]]:
    """Load execution context data for a mission (if available)"""
    file_path = Path(data_dir) / "mission_execution_context.json"

    if not Path(file_path).exists():
        return None

    contexts = load_json_file(str(file_path))
    context_data = next(
        (c for c in contexts if c.get("mission_id") == mission_id),
        None
    )

    return context_data.get("context_data") if context_data else None


def load_agent_result_template(task_id: str, data_dir: str = "agents/data") -> Optional[Dict[str, Any]]:
    """Load agent execution result template for a task (if available)"""
    file_path = Path(data_dir) / "agent_execution_results.json"

    if not Path(file_path).exists():
        return None

    result_templates = load_json_file(str(file_path))
    template = next(
        (r for r in result_templates if r.get("task_id") == task_id),
        None
    )

    return template.get("result_template") if template else None
